# Импорты библиотек

In [1]:
#!pip install gradio pandas plotly matplotlib

import gradio as gr
import pandas as pd
import numpy as np
import gradio as gr
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import warnings

warnings.filterwarnings("ignore")

%load_ext jupyter_black
# Добавьте необходимые библиотеки здесь

# Чтение данных

In [2]:
df_path = "app_survey.csv"

df = pd.read_csv(df_path)

df.head()

,user_id,survey_creation_dt,survey_response_dt,csat_level,language,age,gender,tenure_years,user_income
0,user_00000,2025-04-28 16:01:51,2025-05-07 03:24:28,1,RU,35-44,M,21.105392,10000-25000
1,user_00001,2025-04-25 03:02:23,2025-04-29 13:12:29,5,RU,35-44,M,21.129793,68000-100000
2,user_00002,2025-04-30 11:02:14,2025-04-20 21:09:01,5,RU,45-54,M,21.098074,NaN
3,user_00003,2025-04-25 09:32:18,2025-04-26 19:51:12,5,RU,45-54,M,21.115544,25000-44000
4,user_00004,2025-04-30 08:32:06,2025-05-01 18:33:35,5,RU,45-54,M,21.127961,44000-68000


# Чистка и подготовка данных

In [3]:
df["survey_creation_dt"] = pd.to_datetime(df["survey_creation_dt"])
df["survey_response_dt"] = pd.to_datetime(df["survey_response_dt"])
df["user_id"] = df["user_id"].str[-5:].astype(int)

# Дашборд

In [4]:
# Создание новых признаков
def load_and_preprocess_data():
    df["response_days"] = (df["survey_response_dt"] - df["survey_creation_dt"]).dt.days
    df["is_quick_response"] = df["response_days"] <= 1
    df["tenure_group"] = pd.cut(
        df["tenure_years"],
        bins=[0, 2, 5, 10, 50],
        labels=["0-2г", "3-5л", "6-10л", ">10л"],
    )

    def map_income_group(income):
        if pd.isna(income):
            return "Не указан"
        elif "<" in str(income) or "25000" in str(income):
            return "Низкий (<25K)"
        elif "100000" in str(income) or ">" in str(income):
            return "Высокий (>100K)"
        else:
            return "Средний"

    df["income_group"] = df["user_income"].apply(map_income_group)
    return df


df = load_and_preprocess_data()


# Фильтрация
def apply_filters(df, gender, age, income_group):
    filtered = df.copy()
    if gender != "Все":
        filtered = filtered[filtered["gender"] == gender]
    if age != "Все":
        filtered = filtered[filtered["age"] == age]
    if income_group != "Все":
        filtered = filtered[filtered["income_group"] == income_group]
    return filtered


# Сводная статистика
def get_summary_statistics(filtered_df):
    if len(filtered_df) == 0:
        return "Нет данных для выбранных фильтров"

    stats = []
    stats.append(f"**Количество ответов:** {len(filtered_df):,}")
    stats.append(f"**Средний CSAT:** {filtered_df['csat_level'].mean():.2f}")
    stats.append(f"**Медиана CSAT:** {filtered_df['csat_level'].median():.1f}")
    stats.append(f"**Доля оценок 5:** {(filtered_df['csat_level'] == 5).mean():.1%}")
    stats.append(
        f"**Среднее время ответа:** {filtered_df['response_days'].mean():.1f} дней"
    )
    stats.append(
        f"**Доля быстрых ответов:** {filtered_df['is_quick_response'].mean():.1%}"
    )

    return "<br>".join(stats)


# Визуализация
def plot_satisfaction_by_time(filtered_df):
    daily_stats = (
        filtered_df.groupby(filtered_df["survey_creation_dt"].dt.date)
        .agg({"csat_level": ["mean", "count"], "is_quick_response": "mean"})
        .reset_index()
    )
    daily_stats.columns = ["date", "mean_csat", "response_count", "quick_response_rate"]

    fig = make_subplots(
        rows=3,
        cols=1,
        subplot_titles=(
            "Средняя оценка по дням",
            "Количество ответов",
            "Доля быстрых ответов (<1 дня)",
        ),
        vertical_spacing=0.12,
        shared_xaxes=True,
    )

    fig.add_trace(
        go.Scatter(
            x=daily_stats["date"],
            y=daily_stats["mean_csat"],
            mode="lines+markers",
            name="Средний CSAT",
            line=dict(color="royalblue", width=2),
            marker=dict(size=6),
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Bar(
            x=daily_stats["date"],
            y=daily_stats["response_count"],
            name="Ответов в день",
            marker_color="lightseagreen",
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=daily_stats["date"],
            y=daily_stats["quick_response_rate"] * 100,
            mode="lines",
            name="Быстрые ответы",
            line=dict(color="coral", width=2, dash="dot"),
        ),
        row=3,
        col=1,
    )

    # Убираем сетку и фон
    for row in [1, 2, 3]:
        fig.update_xaxes(showgrid=False, row=row, col=1)
        fig.update_yaxes(showgrid=False, row=row, col=1)

    fig.update_xaxes(title_text="Дата", row=3, col=1)
    fig.update_yaxes(title_text="Средний CSAT", row=1, col=1, range=[1, 5])
    fig.update_yaxes(title_text="Количество", row=2, col=1)
    fig.update_yaxes(title_text="% быстрых ответов", row=3, col=1, range=[0, 100])

    fig.update_layout(
        height=700,
        showlegend=True,
        title_text="Динамика оценок во времени",
        hovermode="x unified",
        plot_bgcolor="white",
        paper_bgcolor="white",
    )
    return fig


def plot_income_tenure_matrix(filtered_df):
    matrix = filtered_df.pivot_table(
        values="csat_level",
        index="income_group",
        columns="tenure_group",
        aggfunc="mean",
        observed=False,
    ).fillna(0)

    fig = px.imshow(
        matrix,
        labels=dict(x="Стаж", y="Группа дохода", color="Средний CSAT"),
        x=matrix.columns,
        y=matrix.index,
        color_continuous_scale="RdYlGn",
        aspect="auto",
        text_auto=".2f",
    )

    # Убираем сетку и фон
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)

    fig.update_layout(
        title="Матрица: средняя оценка по доходу и стажу",
        height=500,
        xaxis_title="Группа стажа",
        yaxis_title="Группа дохода",
        plot_bgcolor="white",
        paper_bgcolor="white",
    )
    return fig


def plot_response_time_analysis(filtered_df):
    response_data = filtered_df[filtered_df["response_days"] <= 30].copy()

    # Используем 2x2 структуру, но помещаем 3 графика в 3 ячейки
    fig = make_subplots(
        rows=2,
        cols=2,
        subplot_titles=(
            "Распределение времени ответа",
            "Средний CSAT по времени ответа",
            "Быстрые vs Медленные ответы",
            None,  # Четвертая ячейка пустая
        ),
        specs=[
            [{"type": "histogram"}, {"type": "bar"}],
            [{"type": "box"}, None],  # Boxplot в (2,1), (2,2) пусто
        ],
        column_widths=[0.5, 0.5],
        row_heights=[0.5, 0.5],
    )

    # График 1: Распределение времени ответа - строка 1, колонка 1
    fig.add_trace(
        go.Histogram(
            x=response_data["response_days"],
            nbinsx=20,
            name="Дней до ответа",
            marker_color="lightblue",
        ),
        row=1,
        col=1,
    )

    # Группируем по скорости ответа
    response_data["response_speed"] = pd.cut(
        response_data["response_days"],
        bins=[-1, 1, 3, 7, 30],
        labels=["<1 дня", "1-3 дня", "4-7 дней", ">7 дней"],
    )
    speed_stats = (
        response_data.groupby("response_speed")["csat_level"].mean().reset_index()
    )

    # График 2: Средний CSAT по времени ответа - строка 1, колонка 2
    fig.add_trace(
        go.Bar(
            x=speed_stats["response_speed"],
            y=speed_stats["csat_level"],
            name="Средний CSAT",
            marker_color="coral",
            width=0.4,  # Делаем столбцы тоньше
        ),
        row=1,
        col=2,
    )

    # График 3: Boxplot сравнение - строка 2, колонка 1 (объединенный)
    response_data["is_quick"] = response_data["response_days"] <= 1
    quick_data = response_data[response_data["is_quick"]]["csat_level"]
    slow_data = response_data[~response_data["is_quick"]]["csat_level"]

    # Создаем один Box с двумя наборами данных
    fig.add_trace(
        go.Box(
            y=quick_data,
            name="Быстрые (<1 дня)",
            marker_color="lightgreen",
            boxmean=True,
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Box(
            y=slow_data,
            name="Медленные (>1 дня)",
            marker_color="lightcoral",
            boxmean=True,
        ),
        row=2,
        col=1,
    )

    # Убираем сетку и фон на всех подграфиках
    for row in [1, 2]:
        for col in [1, 2]:
            if row == 2 and col == 2:  # Пропускаем пустую ячейку (2,2)
                continue
            fig.update_xaxes(showgrid=False, row=row, col=col)
            fig.update_yaxes(showgrid=False, row=row, col=col)

    # Настраиваем оси для каждого графика
    fig.update_xaxes(title_text="Дней до ответа", row=1, col=1)
    fig.update_xaxes(title_text="Скорость ответа", row=1, col=2)
    fig.update_xaxes(title_text="Тип ответа", row=2, col=1)

    fig.update_yaxes(title_text="Количество", row=1, col=1)
    fig.update_yaxes(title_text="Средний CSAT", row=1, col=2, range=[1, 5])
    fig.update_yaxes(title_text="Оценка CSAT", row=2, col=1, range=[1, 5])

    fig.update_layout(
        height=700,
        showlegend=True,
        title_text="Анализ влияния времени ответа на оценку",
        plot_bgcolor="white",
        paper_bgcolor="white",
    )

    return fig


def update_dashboard(analysis_type, gender, age, income_group):
    filtered_df = apply_filters(df, gender, age, income_group)

    if len(filtered_df) < 10:
        empty_fig = go.Figure()
        empty_fig.add_annotation(
            text="Недостаточно данных",
            xref="paper",
            yref="paper",
            x=0.5,
            y=0.5,
            showarrow=False,
            font=dict(size=16, color="gray"),
        )
        empty_fig.update_layout(height=400, plot_bgcolor="white", paper_bgcolor="white")
        return empty_fig, "⚠️ Выберите другие фильтры"

    if analysis_type == "Динамика оценок во времени":
        fig = plot_satisfaction_by_time(filtered_df)
    elif analysis_type == "Матрица доход/стаж":
        fig = plot_income_tenure_matrix(filtered_df)
    elif analysis_type == "Анализ времени ответа":
        fig = plot_response_time_analysis(filtered_df)
    else:
        fig = go.Figure()

    stats = get_summary_statistics(filtered_df)
    return fig, stats


def create_dashboard():
    with gr.Blocks(title="Дашборд: Анализ CSAT", theme=gr.themes.Soft()) as dashboard:
        gr.Markdown("# Анализ удовлетворенности пенсионным приложением")

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### Фильтры")
                gender_filter = gr.Dropdown(["Все", "M", "F"], value="Все", label="Пол")
                age_filter = gr.Dropdown(
                    ["Все"] + sorted(df["age"].dropna().unique().tolist()),
                    value="Все",
                    label="Возраст",
                )
                income_filter = gr.Dropdown(
                    ["Все", "Низкий (<25K)", "Средний", "Высокий (>100K)"],
                    value="Все",
                    label="Доход",
                )

                gr.Markdown("### Тип анализа")
                analysis_type = gr.Radio(
                    choices=[
                        "Динамика оценок во времени",
                        "Матрица доход/стаж",
                        "Анализ времени ответа",
                    ],
                    value="Динамика оценок во времени",
                    label="Визуализация",
                )

                update_btn = gr.Button("Обновить дашборд", variant="primary")

            with gr.Column(scale=2):
                plot_output = gr.Plot()
                stats_output = gr.Markdown()

        # ПРАВИЛЬНОЕ подключение обработчиков событий
        inputs = [analysis_type, gender_filter, age_filter, income_filter]
        outputs = [plot_output, stats_output]

        # Все изменения вызывают update_dashboard напрямую
        update_btn.click(update_dashboard, inputs=inputs, outputs=outputs)
        analysis_type.change(update_dashboard, inputs=inputs, outputs=outputs)
        gender_filter.change(update_dashboard, inputs=inputs, outputs=outputs)
        age_filter.change(update_dashboard, inputs=inputs, outputs=outputs)
        income_filter.change(update_dashboard, inputs=inputs, outputs=outputs)

        # Инициализация при загрузке
        dashboard.load(update_dashboard, inputs=inputs, outputs=outputs)

    return dashboard


# Запуск
if __name__ == "__main__":
    dashboard = create_dashboard()
    dashboard.launch(server_name="0.0.0.0", server_port=7875, share=True)

* Running on local URL:  http://0.0.0.0:7875
* Running on public URL: https://25df293e85c63bb5b8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
